In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.youtube.com/live/H8S9xg8iYuc?si=B_NMiiS7DPvN99-_"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.title.string.replace(" - YouTube", "").strip()

In [2]:
import re

# get video title name
def clean_title(text):
    return re.sub(r'[^0-9a-zA-Z\u0E00-\u0E7F\.]', '', text)
title = clean_title(text=title)

# get video id
match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
if match:
    video_id = match.group(1)

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi

# Fetch transcript (auto-captions or uploaded)
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['th', 'en'])

# Optionally, save to file
file_name = f'{title}_{video_id}'
with open(f"{file_name}_subtitle.txt", "w", encoding="utf-8") as f:
    for entry in transcript:
        f.write(f"{entry['start']:.2f}s: {entry['text']}\n")
        # f.write(f"{entry['text']}")


In [4]:
import re

def load_subtitles(file_path):
    subtitles = []
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            match = re.match(r'([0-9.]+)s:\s(.+)', line.strip())
            if match:
                start_time = float(match.group(1))
                text = match.group(2)
                subtitles.append({'start': start_time, 'text': text})
    return subtitles

def chunk_subtitles(subtitles, chunk_size=60, overlap=20):
    """
    Chunk subtitles into segments of `chunk_size` seconds with `overlap` seconds.
    """
    chunks = []
    max_time = subtitles[-1]['start']
    start_time = 0

    while start_time <= max_time:
        end_time = start_time + chunk_size
        chunk_text = []
        for entry in subtitles:
            if start_time <= entry['start'] < end_time:
                chunk_text.append(entry['text'])
        if chunk_text:
            chunks.append({
                'start': start_time,
                'end': end_time,
                'text': ' '.join(chunk_text)
            })
        start_time += chunk_size - overlap
    return chunks

subtitles = load_subtitles(f"{file_name}_subtitle.txt")
chunk_dict = chunk_subtitles(subtitles, chunk_size=30, overlap=10)

In [5]:
chunks = [chunk['text'] for chunk in chunk_dict]

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# Embed text
embeddings = embedding_model.encode(chunks)
print(f"Vector length: {len(embeddings[0])}")

c:\Users\terjr\miniconda3\envs\ollama_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import chromadb

# Local vector DB (PersistentClient = new style!)
chroma_client = chromadb.PersistentClient(path="./vector_database")

collection = chroma_client.get_or_create_collection(name="my_local_collection")

In [8]:
# Cloud collections work the same way!
collection.add(
    ids=[f"doc_{i}" for i in range(len(chunks))],  # unique IDs
    embeddings=embeddings.tolist(),               # must be list of lists!
    documents=chunks,                              # optional, but useful
    metadatas=[{"source": "example"} for _ in chunks]  # optional metadata
)

In [9]:
title = file_name.split('_')[0]

In [23]:
query = "what are the growth driven strategy for the future?"
# query = """Analyze the company's revenue and profit performance over the past 4 quarters.
# 1. Compare YoY (year-over-year) and QoQ (quarter-over-quarter) changes.
# 2. Check whether the actual results met, exceeded, or missed previous guidance.
# 3. Highlight any unexpected spikes, drops, or anomalies.
# 4. If possible, explain the reasons behind these surprises based on management's comments."""

# Embed locally
query_embedding = embedding_model.encode([query])

# Search Chroma Cloud
results = collection.query(
    query_embeddings=query_embedding.tolist(),
    n_results=10
)

# Extract relevant chunks
contexts = results['documents'][0]
context_text = "\n".join(contexts)

In [20]:
contexts

['อย่างเช่นเอ่อแตงกวาดองหรือว่าที่สำคัญ เนี่ยที่ร้านไก่โจวิงเนี่ยคือเรายังแบบ หยิบตัวหลักของเราที่เป็นสลัดออร์แกนิค อ่ะมา develop เพิ่มเติมเพราะว่าบางทีอ่ะ เอ่ออย่างอย่าผมผมไปอู่ไปเนี่ยเราเป็นพ่อ ใช่มั้ยบางทีเราก็สามารถกินสลัดได้อยู่ ส่วนลูกๆที่ชอบกินไข่เงี้ยก็ก็กินไก่ไป อะไรอย่างเงี้ยซึ่งเมนูที่เราเอ่อ Develop รสชาติมาเนี่ยมันจะมีแบบเผ็ดๆสุดๆก็อาจจะ เป็นผู้ใหญ่ทานใช่มั้ครับแต่ว่าถ้าเผ็ด',
 ' 10 ของซอที่ขายดีด้วยนะครับก็ทั้งนี้ ทั้งนั้นเ่อในสิ่งที่เราทำสำหรับโจงเราก็ มองเป็นโอกาสอยู่ 3 ด้านหลักๆเนาะอันแรก เนี่ยเราคิดว่าแบรนด์เนี้ยตอบโจทย์กลุ่ม ลูกค้าเจนซีซึ่งเป็นกลุ่มนักเรียนนัก ศึกษากลุ่มกลุ่มวัยทำงานที่อายุประมาณ 10 ต้นๆถึง 20 ปลายๆนะครับก็จะเป็นอีกทarget กรุ๊ปนึงที่เพิ่มเติมจากโอกาจู่ซึ่ง โอ๊กจู่อาจจะแบบเป็นกลุ่มครอบครัวแล้วก็ มีอายุมากขึ้นตรงนี้เองแล้วก็ตอบโจทย์ กลุ่มที่อายุน้อยลงนะครับแล้วก็ข้อที่ 2 เนี่ยคือเหมือนว่าเราก็ develop ควบคู่',
 'ยังมองว่าเราก็ยังเติบโตได้ทั้งในส่วนของ ตัวโอ้กจิ๋วเองและแบรนด์ใหม่ๆของเราที่ เราก็เปิดอยู่นะครับก็แน่นอนครับเราก็อาจ จะต้องเปิดสาขาแบบที่ระมั

In [25]:
import ollama

system_prompt = f"""
You are a financial analyst specializing in Thai stock market Opportunity Day (Opp Day) presentations. 
Your role is to carefully analyze company transcripts, investor Q&A, and management comments to extract meaningful insights.
Focus on detecting trends in revenue, profit, margin changes, cost structures, and management guidance.
Highlight any signals of growth, risk, hidden problems, or strategic changes.
Always cross-check the management's statements with financial performance.
Be objective, critical, and concise.
When relevant, summarize both hard data (numbers) and soft signals (tone, confidence, body language hints, or evasive answers).
Output clear, actionable insights and note any red flags or questions that need deeper follow-up.
Your answers must be structured, clear, and suitable for an investor deciding whether to buy, hold, or sell the stock.
"""

user_prompt = f"""Answer the question below using ONLY the context below.

Context:
{context_text}

Question:
{query}

Answer:"""


response = ollama.chat(
    model="llama3",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

summary = response["message"]["content"]

print(summary)

Based on the context, it appears that the company is focusing on three main areas of opportunity:

1. Brand expansion: The company believes its brand can respond to the needs of a new target group, such as young adults and families.
2. Organic growth: They aim to develop their own organic growth strategy, which includes expanding existing locations and opening new ones in strategic locations.
3. Collaboration and innovation: The company sees opportunities for collaboration with other brands and partners to drive growth and innovation.

In terms of specific growth-driven strategies, the CEO mentioned that they will focus on developing their brand's presence among younger demographics (ages 10-20) and expanding into new markets. They also aim to open more locations in strategic areas, while maintaining a selective approach to ensure ROI is maintained.

Additionally, the CFO mentioned that they are monitoring risk management closely, ensuring that the company has a robust system in place 